## **| 앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**

직접 투표 분류기는 횟수를 중요시하여, 가장 많은 투표를 얻은 클래스를 선택. (다수결)

간접 투표 분류기는 각 클래스의 평균적인 확률 추정값을 계산해서 가장 높은 확률을 가진 클래스를 고름. 이 방법은 신뢰가 높은 투표에 더 가중치를 주고 종종 더 나은 성능을 낸다. 하지만 앙상블에 있는 모든 분류기가 클래스 확률을 추정할 수 있어야 사용 가능!

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

학습률을 감소시켜야한다 ( 예측기 수가 너무 많으면 ) 알맞은 개수를 찾기 위해 조기 종료 기법을 사용할 수 있다.

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [5]:

from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

In [7]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다.

In [8]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [9]:
# model fitting

rnd_clf = RandomForestClassifier(n_estimators=10,random_state=0)
ext_clf = ExtraTreesClassifier(n_estimators=10, random_state=0)
svm_clf = LinearSVC(max_iter=10000,random_state=0)
mlp_clf = MLPClassifier(random_state=0)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [10]:
from sklearn.metrics import accuracy_score

In [12]:
# model fitting
models = [rnd_clf,ext_clf,svm_clf,mlp_clf]

for model in models:
    print("훈련 :",model)
    model.fit(X_train,y_train)


훈련 : RandomForestClassifier(n_estimators=10, random_state=0)
훈련 : ExtraTreesClassifier(n_estimators=10, random_state=0)
훈련 : LinearSVC(max_iter=10000, random_state=0)


C:\Users\MYCOM\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


훈련 : MLPClassifier(random_state=0)


In [13]:
# model test

for model in models:
    y_pred = model.predict(X_test)
    print("정확도 :",accuracy_score(y_test,y_pred))

정확도 : 0.9429047619047619
정확도 : 0.9484761904761905
정확도 : 0.8571904761904762
정확도 : 0.9654761904761905


####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [ ]:
# 각 분류 모델 학습
from sklearn.ensemble import VotingClassifier

named_models = [
    ("RandomForest" , rnd_clf),
    ("ExtraTree" , ext_clf),
    ("LinearSVM" , svm_clf),
    ("MLP" , mlp_clf),
]
voting_clf = VotingClassifier(named_models)
voting_clf.fit(X_train,y_train)
voting_clf.set_params(LinearSVM=None)

- Q. 어떤 모델의 성능이 가장 낮나요?
- A.SVM

In [ ]:
# 가장 성능이 낮은 모델 제거

voting_clf.set_params(LinearSVM=None)

In [ ]:
# 모델 제거 후 성능 확인

voting_clf.score(X_test,y_test)

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [ ]:
import numpy as np
X_test_pred = np.empty((len(X_test), len(models)), dtype=np.float32)
for i, model in enumerate(models):
    X_test_pred[:,i] = model.predict(X_test)
X_test_pred


####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [ ]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200,oob_score=True,random_state=42)
rnd_forest_blender.fit(X_test_pred,y_test)

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다.

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [ ]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
X_test_pred = np.empty((len(X_test), len(models)), dtype=np.float32)

for i, model in enumerate(models):
    X_test_pred[:,i] = model.predict(X_test)  # 각 리스트에서 i열만 뽑아내기 

In [ ]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
y_pred = rnd_forest_blender.predict(X_test_pred)

In [ ]:
# model test
accuracy_score(y_test,y_pred)